## 직렬화(Serialization)

### 직렬화(Serialization) 란?

1. **정의:**
   - 모델을 저장 가능한 형식으로 변환하는 과정

2. **목적:**
   - 모델 재사용 (재훈련 없이)
   - 모델 배포 및 공유 용이
   - 계산 리소스 절약

3. **장점:**
   - 빠른 모델 로딩
   - 버전 관리 가능
   - 다양한 환경에서 사용 가능

모델 직렬화는 AI 개발 및 배포 과정에서 중요한 단계로, 효율적인 모델 관리와 재사용을 가능하게 합니다.

`is_lc_serializable` 클래스 메서드로 실행하여 LangChain 클래스가 직렬화 가능한지 확인할 수 있습니다.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [1]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


In [3]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{fruit}의 색상이 무엇입니까?")

In [4]:
print(f"ChatOpenAI: {ChatOpenAI.is_lc_serializable()}")

ChatOpenAI: True


In [5]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

print(f"ChatOpenAI: {llm.is_lc_serializable()}")

ChatOpenAI: True


In [6]:
chain = prompt | llm
chain.is_lc_serializable()

True

## 체인(Chain) 직렬화(dumps, dumpd)

### 개요

체인 직렬화는 직렬화 가능한 모든 객체를 딕셔너리 또는 JSON 문자열로 변환하는 과정을 의미합니다.

### 직렬화 방법

객체의 속성 및 데이터를 키-값 쌍으로 저장하여 딕셔너리 형태로 변환합니다.

이러한 직렬화 방식은 객체를 쉽게 저장하고 전송할 수 있게 하며, 다양한 환경에서 객체를 재구성할 수 있도록 합니다.

**참고**
- `dumps`: 객체를 JSON 문자열로 직렬화
- `dumpd`: 객체를 딕셔너리로 직렬화


In [7]:
from langchain_core.load import dumpd, dumps
dumpd_chain = dumpd(chain)
dumpd_chain

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'prompts', 'prompt', 'PromptTemplate'],
   'kwargs': {'input_variables': ['fruit'],
    'template': '{fruit}의 색상이 무엇입니까?',
    'template_format': 'f-string'},
   'name': 'PromptTemplate'},
  'last': {'lc': 1,
   'type': 'constructor',
   'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
   'kwargs': {'model_name': 'gpt-4o-mini',
    'temperature': 0.0,
    'openai_api_key': {'lc': 1, 'type': 'secret', 'id': ['OPENAI_API_KEY']}},
   'name': 'ChatOpenAI'}},
 'name': 'RunnableSequence'}

In [8]:
type(dumpd_chain)

dict

## Pickle 파일

### 개요

Pickle 파일은 Python 객체를 바이너리 형태로 직렬화하는 포맷입니다.

### 특징

1. **형식:**
   - Python 객체를 바이너리 형태로 직렬화하는 포맷

2. **특징:**
   - Python 전용 (다른 언어와 호환 불가)
   - 대부분의 Python 데이터 타입 지원 (리스트, 딕셔너리, 클래스 등)
   - 객체의 상태와 구조를 그대로 보존

3. **장점:**
   - 효율적인 저장 및 전송
   - 복잡한 객체 구조 유지
   - 빠른 직렬화/역직렬화 속도

4. **단점:**
   - 보안 위험 (신뢰할 수 없는 데이터 역직렬화 시 주의 필요)
   - 사람이 읽을 수 없는 바이너리 형식

### 주요 용도

1. 객체 캐싱
2. 머신러닝 모델 저장
3. 프로그램 상태 저장 및 복원

### 사용법

- `pickle.dump()`: 객체를 파일에 저장
- `pickle.load()`: 파일에서 객체 로드


In [9]:
import pickle

# fuit_chain.pkl 파일로 직렬화된 체인을 저장합니다.
with open("fruit_chain.pkl", "wb") as f:
    pickle.dump(dumpd_chain, f)

In [10]:
import json

with open('fruit_chain.json', 'w') as fp:
    json.dump(dumpd_chain, fp)

In [11]:
# pickle 파일을 로드합니다.
with open("fruit_chain.pkl", "rb") as f:
    loaded_chain = pickle.load(f)

In [12]:
from langchain_core.load import load

# 체인을 로드합니다.
chain_from_file = load(loaded_chain)

# 체인을 실행합니다.
print(chain_from_file.invoke({"fruit": "사과"}))

C:\Users\sbj57\AppData\Local\Temp\ipykernel_23328\741098595.py:4: LangChainBetaWarning: The function `load` is in beta. It is actively being worked on, so the API may change.
  chain_from_file = load(loaded_chain)


content='사과의 색상은 품종에 따라 다양합니다. 일반적으로 사과는 빨간색, 초록색, 노란색 등 여러 가지 색상을 가질 수 있습니다. 예를 들어, 레드 딜리셔스는 주로 빨간색이고, 그라니 스미스는 초록색이며, 골든 딜리셔스는 노란색입니다. 또한, 일부 사과는 두 가지 이상의 색상을 조합한 경우도 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 16, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BI6SsCApKro0UmF8fIJgjaRmV0Vca', 'finish_reason': 'stop', 'logprobs': None} id='run-d62b5a80-8320-4f5f-8f9b-cd9ab0f3213f-0' usage_metadata={'input_tokens': 16, 'output_tokens': 102, 'total_tokens': 118, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [13]:
from langchain_core.load import load, loads

load_chain = load(
    loaded_chain, secrets_map={"OPENAI_API_KEY": os.environ["OPENAI_API_KEY"]}
)

# 불러온 체인이 정상 동작하는지 확인합니다.
load_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 다양합니다. 일반적으로 빨간색, 초록색, 노란색 등 여러 가지 색상이 있으며, 품종에 따라 다르게 나타납니다. 예를 들어, 레드 딜리셔스는 주로 빨간색이고, 그라니 스미스는 초록색이며, 골든 딜리셔스는 노란색입니다. 또한, 일부 사과는 두 가지 이상의 색상이 섞여 있는 경우도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 16, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BI6TOCcly95yzxStk4VoAA4jq2TRI', 'finish_reason': 'stop', 'logprobs': None}, id='run-621ebe51-e631-49f9-8166-cf02651e00c5-0', usage_metadata={'input_tokens': 16, 'output_tokens': 103, 'total_tokens': 119, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
with open("fruit_chain.json", "r") as fp:
    loaded_from_json_chain = json.load(fp)
    loads_chain = load(loaded_from_json_chain)

In [15]:
# 불러온 체인이 정상 동작하는지 확인합니다.
loads_chain.invoke({"fruit": "사과"})

AIMessage(content='사과의 색상은 다양합니다. 일반적으로 빨간색, 초록색, 노란색 등 여러 가지 색상이 있으며, 품종에 따라 다르게 나타납니다. 예를 들어, 레드 딜리셔스는 주로 빨간색이고, 그라니 스미스는 초록색이며, 골든 딜리셔스는 노란색입니다. 또한, 일부 사과는 두 가지 이상의 색상이 섞여 있는 경우도 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 16, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b376dfbbd5', 'id': 'chatcmpl-BI6TVp3quaXl8S1PIQNEoz0NLfDLu', 'finish_reason': 'stop', 'logprobs': None}, id='run-e26f96d8-3936-446c-867c-6442381efb06-0', usage_metadata={'input_tokens': 16, 'output_tokens': 103, 'total_tokens': 119, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})